<a href="https://colab.research.google.com/github/ykitaguchi77/GAN/blob/master/Toony_Yourself.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Toony Yourself**

implementation: http://cedro3.com/ai/toonify-yourself/#comment-869

original: 

In [2]:
#StyleGANをGit clone
%tensorflow_version 1.x
!git clone https://github.com/justinpinkney/stylegan2

%cd stylegan2
!nvcc test_nvcc.cu -o test_nvcc -run
 
# フォルダーの作成
!mkdir aligned
!mkdir generated

TensorFlow 1.x selected.
Cloning into 'stylegan2'...
remote: Enumerating objects: 269, done.
remote: Total 269 (delta 0), reused 0 (delta 0), pack-reused 269
Receiving objects: 100% (269/269), 2.32 MiB | 12.30 MiB/s, done.
Resolving deltas: 100% (141/141), done.
/content/stylegan2
CPU says hello.
GPU says hello.


In [3]:
# ffhqモデル(Baseモデル)とblendedモデルのダウンロード
#import pretrained_networks
import gdown

 
# use my copy of the blended model to save Doron's download bandwidth
# get the original here https://mega.nz/folder/OtllzJwa#C947mCCdEfMCRTWnDcs4qw
#blended_url = "https://drive.google.com/uc?id=1H73TfV5gQ9ot7slSed_l-lim9X7pMRiU"
blended_url = "https://drive.google.com/uc?id=1BRqqHWk_4BjNHLXTrpHkoxmZhGw3CU59" 
ffhq_url = "http://d36zk2xti64re0.cloudfront.net/stylegan2/networks/stylegan2-ffhq-config-f.pkl"

#エラーがでてうまくいかないので、ダイレクトにダウンロードすることにする 
#_, _, Gs_blended = pretrained_networks.load_networks(blended_url)
#_, _, Gs = pretrained_networks.load_networks(ffhq_url)
gdown.download(blended_url, quiet=False)
gdown.download(ffhq_url, quiet=False)


Downloading...
From: https://drive.google.com/uc?id=1BRqqHWk_4BjNHLXTrpHkoxmZhGw3CU59
To: /content/stylegan2/ffhq-cartoon-blended-64.pkl
100%|██████████| 382M/382M [00:01<00:00, 230MB/s]
Downloading...
From: http://d36zk2xti64re0.cloudfront.net/stylegan2/networks/stylegan2-ffhq-config-f.pkl
To: /content/stylegan2/stylegan2-ffhq-config-f.pkl
100%|██████████| 382M/382M [00:01<00:00, 346MB/s]


'stylegan2-ffhq-config-f.pkl'

#**必要な関数を定義**

In [4]:
# 画像表示
import matplotlib.pyplot as plt
from PIL import Image
import glob
import numpy as np

def display_pic(folder):
    fig = plt.figure(figsize=(30, 40))
    files = glob.glob(folder)
    files.sort()
    images=[]
    for i in range(len(files)):
        img = Image.open(files[i])    
        images = np.asarray(img)
        ax = fig.add_subplot(10, 10, i+1, xticks=[], yticks=[])
        image_plt = np.array(images)
        ax.imshow(image_plt)
        ax.set_xlabel(str(i), fontsize=20)               
    plt.show()
    plt.close()  

# 潜在変数(latents)から生成した画像を表示
import PIL.Image
def display(latents):
    
    synthesis_kwargs = dict(output_transform=dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=False), minibatch_size=8)
  
    fig = plt.figure(figsize=(30, 40))   
    for i in range(len(latents)):
        vec = latents[i].reshape(1,18,512)
        images =  Gs_blended.components.synthesis.run(vec, randomize_noise=False, **synthesis_kwargs)  
        images = images.transpose((0,2,3,1))     
        PIL.Image.fromarray(images[0], 'RGB')   
        ax = fig.add_subplot(10, 10, i+1, xticks=[], yticks=[])
        image_plt = np.array(images[0])
        ax.imshow(image_plt)
        ax.set_xlabel(str(i), fontsize=20)               
    plt.show()
    plt.close()

# 潜在変数(latents)の順番を指定して、トランジションのGIFを作成する
import os

def generate_gif(latents, idx):

    synthesis_kwargs = dict(output_transform=dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=False), minibatch_size=8)

    image_gif = []
    os.makedirs('my/gif', exist_ok=True)
    for j in range(len(idx)-1):
        for i in range(20):
            latent = latents[idx[j]]+(latents[idx[j+1]]-latents[idx[j]])*i/19
            latent = latent.reshape(1, 18, 512)
            images =  Gs_blended.components.synthesis.run(latent, randomize_noise=False, **synthesis_kwargs) 
            images = images.transpose((0,2,3,1)) 
            image_one = PIL.Image.fromarray(images[0], 'RGB')
            image_gif.append(image_one.resize((256,256))) 

    image_gif[0].save('./my/gif/anime.gif', save_all=True, append_images=image_gif[1:],
                      duration=100, loop=0) 

In [5]:
# サンプル写真のダウンロードと整形
import os 
import bz2
import shutil
import requests
import urllib.request


# rawフォルダを作成
try: 
    shutil.rmtree("/content/stylegan2/raw")
    print("directory deleted")
    print("")
except:
    pass
os.makedirs("/content/stylegan2/raw", exist_ok=True)


#Webから画像をダウンロードしてrawフォルダに格納
url = "https://images.unsplash.com/photo-1597223557154-721c1cecc4b0?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHxzZWFyY2h8Mnx8aHVtYW4lMjBmYWNlfGVufDB8fDB8fA%3D%3D&w=1000&q=80"
file_name = "/content/stylegan2/raw/sample.jpg"

response = requests.get(url)
image = response.content
with open(file_name, "wb") as aaa:
    aaa.write(image)

"""
#dlibのshape_predictor_5_face_landmarks.datをダウンロード
url='http://dlib.net/files/shape_predictor_5_face_landmarks.dat.bz2'
save_name='shape_predictor_5_face_landmarks.dat.bz2'
urllib.request.urlretrieve(url, save_name)
"""

"\n#dlibのshape_predictor_5_face_landmarks.datをダウンロード\nurl='http://dlib.net/files/shape_predictor_5_face_landmarks.dat.bz2'\nsave_name='shape_predictor_5_face_landmarks.dat.bz2'\nurllib.request.urlretrieve(url, save_name)\n"

In [ ]:
!pip install mediapipe opencv-python

import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh

#サポートパッチのインポート
from google.colab.patches import cv2_imshow


# For static images:
IMAGE_FILES = ['/content/stylegan2/raw/sample.jpg']
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
with mp_face_mesh.FaceMesh(
    static_image_mode=True,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5) as face_mesh:
  for idx, file in enumerate(IMAGE_FILES):
    image = cv2.imread(file)
    # Convert the BGR image to RGB before processing.
    results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    # Print and draw face mesh landmarks on the image.
    if not results.multi_face_landmarks:
      continue
    annotated_image = image.copy()
    for face_landmarks in results.multi_face_landmarks:
      #print('face_landmarks:', face_landmarks)
      
      mp_drawing.draw_landmarks(
          image=annotated_image,
          landmark_list=face_landmarks,
          connections=mp_face_mesh.FACEMESH_TESSELATION,
          landmark_drawing_spec=None,
          connection_drawing_spec=mp_drawing_styles
          .get_default_face_mesh_tesselation_style())
      # mp_drawing.draw_landmarks(
      #     image=annotated_image,
      #     landmark_list=face_landmarks,
      #     connections=mp_face_mesh.FACEMESH_CONTOURS,
      #     landmark_drawing_spec=None,
      #     connection_drawing_spec=mp_drawing_styles
      #     .get_default_face_mesh_contours_style())
      # mp_drawing.draw_landmarks(
      #     image=annotated_image,
      #     landmark_list=face_landmarks,
      #     connections=mp_face_mesh.FACEMESH_IRISES,
      #     landmark_drawing_spec=None,
      #     connection_drawing_spec=mp_drawing_styles
      #     .get_default_face_mesh_iris_connections_style())
    cv2.imwrite('/tmp/annotated_image' + str(idx) + '.png', annotated_image)
    
cv2_imshow(annotated_image)

In [33]:
image = cv2.imread(file)
image.shape

(1000, 1000, 3)

In [27]:
landmarks = results.multi_face_landmarks[0]

In [29]:
landmarks.landmark[0]

x: 0.5013298392295837
y: 0.729648768901825
z: -0.05366979166865349

In [5]:
import cv2
import numpy as np
from PIL import Image
import dlib
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_5_face_landmarks.dat')

img = url

def shape_to_normal(shape):
    shape_normal = []
    for i in range(0, 5):
        shape_normal.append((i, (shape.part(i).x, shape.part(i).y)))
    return shape_normal
def get_eyes_nose_dlib(shape):
    nose = shape[4][1]
    left_eye_x = int(shape[3][1][0] + shape[2][1][0]) // 2
    left_eye_y = int(shape[3][1][1] + shape[2][1][1]) // 2
    right_eyes_x = int(shape[1][1][0] + shape[0][1][0]) // 2
    right_eyes_y = int(shape[1][1][1] + shape[0][1][1]) // 2
    return nose, (left_eye_x, left_eye_y), (right_eyes_x, right_eyes_y)

def distance(a, b):
    return np.sqrt((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2)

def cosine_formula(length_line1, length_line2, length_line3):
    cos_a = -(length_line3 ** 2 - length_line2 ** 2 - length_line1 ** 2) / (2 * length_line2 * length_line1)
    return cos_a

def rotate_point(origin, point, angle):
    ox, oy = origin
    px, py = point

    qx = ox + np.cos(angle) * (px - ox) - np.sin(angle) * (py - oy)
    qy = oy + np.sin(angle) * (px - ox) + np.cos(angle) * (py - oy)
    return qx, qy


def is_between(point1, point2, point3, extra_point):
    c1 = (point2[0] - point1[0]) * (extra_point[1] - point1[1]) - (point2[1] - point1[1]) * (extra_point[0] - point1[0])
    c2 = (point3[0] - point2[0]) * (extra_point[1] - point2[1]) - (point3[1] - point2[1]) * (extra_point[0] - point2[0])
    c3 = (point1[0] - point3[0]) * (extra_point[1] - point3[1]) - (point1[1] - point3[1]) * (extra_point[0] - point3[0])
    if (c1 < 0 and c2 < 0 and c3 < 0) or (c1 > 0 and c2 > 0 and c3 > 0):
        return True
    else:
        return False

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
rects = detector(gray, 0)
if len(rects) > 0:
    for rect in rects:
        x = rect.left()
        y = rect.top()
        w = rect.right()
        h = rect.bottom()
        shape = predictor(gray, rect)

#obtain central coordinates of nose and eyes
shape = shape_to_normal(shape)
nose, left_eye, right_eye = get_eyes_nose_dlib(shape)
center_of_forehead = ((left_eye[0] + right_eye[0]) // 2, (left_eye[1] + right_eye[1]) // 2)
center_pred = (int((x + w) / 2), int((y + y) / 2))

length_line1 = distance(center_of_forehead, nose)
length_line2 = distance(center_pred, nose)
length_line3 = distance(center_pred, center_of_forehead)

cos_a = cosine_formula(length_line1, length_line2, length_line3)
angle = np.arccos(cos_a)

rotated_point = rotate_point(nose, center_of_forehead, angle)
rotated_point = (int(rotated_point[0]), int(rotated_point[1]))
if is_between(nose, center_of_forehead, center_pred, rotated_point):
    angle = np.degrees(-angle)
else:
    angle = np.degrees(angle)

img = Image.fromarray(img)
img = np.array(img.rotate(angle))


RuntimeError: ignored

In [ ]:


# 顔画像の切り取り
!python align_images.py raw aligned
display_pic('./aligned/*.png')

Traceback (most recent call last):
  File "align_images.py", line 33, in <module>
    for img_name in [x for x in os.listdir(RAW_IMAGES_DIR) if x[0] not in '._']:
FileNotFoundError: [Errno 2] No such file or directory: 'raw'


<Figure size 2160x2880 with 0 Axes>